## В этом домашнем задании вы сделаете первые шаги в мире линейной бинарной классификации!

In [2]:
import pandas as pd
import numpy as np
import warnings



pd.options.display.max_columns = 500

In [3]:
# Имя исходного файла и файла для сохранения
input_file = "train.csv"
output_file = "train_1.csv"

# Лимит строк для сохранения
row_limit = 200000

# Чтение и запись строк
with open(input_file, "r", encoding="utf-8") as infile, open(output_file, "w", encoding="utf-8") as outfile:
    # Считаем строки
    for i, line in enumerate(infile):
        if i == row_limit:
            break  # Если достигли лимита, остановить чтение
        outfile.write(line)  # Записываем строку в новый файл

print(f"Первые {row_limit} строк сохранены в файл {output_file}.")


Первые 200000 строк сохранены в файл train_1.csv.


#### Задание 1

Мы будем работать с данными **Microsoft Malware Detection**

Таргетом будет последний столбец `HasDetection`, который принимает значения $\{0,\, 1\}$ в случае отсутствия или наличия вируса на компьютере соответственно. Признаками будут выступать всевозможные характеристики коспьютера.

In [4]:
### Загрузим датасет

file_path_1 = 'train_1.csv'


# Укажите явные типы данных для проблемных столбцов
dtype = {
    'AVProductStatesIdentifier': 'float64',
    'AVProductsEnabled': 'float64',
    'AVProductsInstalled': 'float64',
    'Census_FirmwareManufacturerIdentifier': 'float64',
    'Census_FirmwareVersionIdentifier': 'float64',
    'Census_InternalBatteryNumberOfCharges': 'float64',
    'Census_InternalPrimaryDisplayResolutionHorizontal': 'float64',
    'Census_InternalPrimaryDisplayResolutionVertical': 'float64',
    'Census_IsAlwaysOnAlwaysConnectedCapable': 'float64',
    'Census_IsFlightsDisabled': 'float64',
    'Census_IsVirtualDevice': 'float64',
    'Census_OEMModelIdentifier': 'float64',
    'Census_OEMNameIdentifier': 'float64',
    'Census_OSInstallLanguageIdentifier': 'float64',
    'Census_PrimaryDiskTotalCapacity': 'float64',
    'Census_ProcessorClass': 'object',  # Столбец, который содержит строки
    'Census_ProcessorCoreCount': 'float64',
    'Census_ProcessorManufacturerIdentifier': 'float64',
    'Census_ProcessorModelIdentifier': 'float64',
    'Census_SystemVolumeTotalCapacity': 'float64',
    'Census_TotalPhysicalRAM': 'float64',
    'CityIdentifier': 'float64',
    'Firewall': 'float64',
    'GeoNameIdentifier': 'float64',
    'IsProtected': 'float64',
    'PuaMode': 'object',  # Столбец, который содержит строки
    'RtpStateBitfield': 'float64',
    'SMode': 'float64',
    'UacLuaenable': 'float64',
    'Wdft_IsGamer': 'float64',
    'Wdft_RegionIdentifier': 'float64'
}

# Загружаем данные с указанными типами

df_1 = pd.read_csv(file_path_1, dtype=dtype)

# Пример обработки данных
df_1.shape


(199999, 83)

In [5]:
import gc
df_1.drop(columns=['ProductName', 'MachineIdentifier'], inplace=True)
df_1 = df_1.loc[:, (df_1.nunique() > 1)]
gc.collect()

0

Удалите константные признаки и признаки `ProductName` `MachineIdentifier`

Посмотрите на соотношение классов в таргете. Все ли хорошо?

In [6]:
### Посмотрим, сколько наблюдений(таргет) в датасете 
### имеют 0 и 1 классы
print(sum(df_1['HasDetections'] == 1), 'positive')
print(sum(df_1['HasDetections'] == 0), 'negative')


### Кажется, с балансом классов все ок!

100059 positive
99940 negative


Ответьте на вопрос: почему с вашей точки зрения важно иметь представление о балансе классов в ваших данных?

**При дисбалансе модели могут подстраиваться под доминирующий класс, когда для нас может быть интересен как раз-таки другой**

Избавьтесь от пропусков в данных! 

Новый для нас прием: признаки с более чем половиной пропусков следует удалить.

Согласитесь, если в вашей колоночке среди 100 объектов всего лишь у 2 есть какое-то непропущенное значение, странно все остальные заполнять средним от этих двух чисел. Такие "редкие" признаки лучше вообще опустить!


В категориальных колонках заменим отсутствующую категорию просто некоторой новой и назовем ее `NaN`.

А в числовых, ради разнообразия, заполним пропуски медианным значением.

In [7]:
### Посчитаем количество отсутствующих значений в каждой колонке
nans = {}
for i in df_1.columns:
    nans[i] = sum(df_1[i].isna())

nans



{'EngineVersion': 0,
 'AppVersion': 0,
 'AvSigVersion': 0,
 'RtpStateBitfield': 764,
 'IsSxsPassiveMode': 0,
 'DefaultBrowsersIdentifier': 190296,
 'AVProductStatesIdentifier': 804,
 'AVProductsInstalled': 804,
 'AVProductsEnabled': 804,
 'HasTpm': 0,
 'CountryIdentifier': 0,
 'CityIdentifier': 7215,
 'OrganizationIdentifier': 61536,
 'GeoNameIdentifier': 1,
 'LocaleEnglishNameIdentifier': 0,
 'Platform': 0,
 'Processor': 0,
 'OsVer': 0,
 'OsBuild': 0,
 'OsSuite': 0,
 'OsPlatformSubRelease': 0,
 'OsBuildLab': 1,
 'SkuEdition': 0,
 'IsProtected': 799,
 'AutoSampleOptIn': 0,
 'SMode': 11863,
 'IeVerIdentifier': 1362,
 'SmartScreen': 71123,
 'Firewall': 2078,
 'UacLuaenable': 248,
 'Census_MDC2FormFactor': 0,
 'Census_DeviceFamily': 0,
 'Census_OEMNameIdentifier': 2082,
 'Census_OEMModelIdentifier': 2245,
 'Census_ProcessorCoreCount': 922,
 'Census_ProcessorManufacturerIdentifier': 922,
 'Census_ProcessorModelIdentifier': 923,
 'Census_ProcessorClass': 199126,
 'Census_PrimaryDiskTotalCap

In [8]:
### Удалим те колонки, в которых пропусков более половины от размера всей выборки
thershold = len(df_1)//2
for i in nans.keys():
    if nans[i] >= thershold:
        df_1.drop(columns=i,inplace=True)
# Your code is here
del thershold
gc.collect()

0

In [9]:
### В категориальных колонках с отсутствующими значениями
### произведем fillna, создав по новой категории для каждой
### такой фичи
cat_col = df_1.select_dtypes(exclude=['float64','int64']).isnull().any()

for null_cal in cat_col[cat_col].index:
    df_1[null_cal].fillna('NaN',inplace=True)
del cat_col
gc.collect()

/tmp/ipykernel_47559/2266193294.py:7: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_1[null_cal].fillna('NaN',inplace=True)


0

In [10]:
### Оставшиеся (вещественные) колонки заполним медианой по столбцу
nans = np.sum(df_1.isna(), axis=0)


for col in list(nans[nans > 0].index):
    df_1[col].fillna(df_1[col].median(), inplace=True)
# Your code is here
del nans
gc.collect()

/tmp/ipykernel_47559/2377439014.py:6: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_1[col].fillna(df_1[col].median(), inplace=True)
/tmp/ipykernel_47559/2377439014.py:6: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', 

0

In [11]:
df_1.shape

(199999, 73)

Создайте копию полученного датафрейма и положите ее в переменную data_2. Понадобится в следующих заданиях.

In [12]:
df_2 = df_1.copy()

Так же поработаем над всеми категориальными колонкам перед запуском непосредственно моделей.

Провернем самый базовый и наглый метод - несмотря на количество уникальных значений в каждой категории, просто применим ко всей категориальной части датасета `OneHotEncoding`

In [13]:


# 1. Выбор категориальных переменных
cat_col = df_1.select_dtypes(exclude=['float64', 'int64']).columns

# 2. Использование OneHotEncoder с sparse матрицей

df_1_cat = pd.get_dummies(data=df_1[cat_col],
                          prefix=cat_col,
                          drop_first=True)

# 3. Удаление исходных категориальных столбцов
df_1 = df_1.drop(cat_col, axis=1)

# 4. Объединение данных (конкатенация с разреженной матрицей)
df_1 = pd.concat((df_1, df_1_cat), axis=1)

df_1.shape


(199999, 6114)

In [14]:
df_1.head(1)

,RtpStateBitfield,IsSxsPassiveMode,AVProductStatesIdentifier,AVProductsInstalled,AVProductsEnabled,HasTpm,CountryIdentifier,CityIdentifier,OrganizationIdentifier,GeoNameIdentifier,LocaleEnglishNameIdentifier,OsBuild,OsSuite,IsProtected,AutoSampleOptIn,SMode,IeVerIdentifier,Firewall,UacLuaenable,Census_OEMNameIdentifier,Census_OEMModelIdentifier,Census_ProcessorCoreCount,Census_ProcessorManufacturerIdentifier,Census_ProcessorModelIdentifier,Census_PrimaryDiskTotalCapacity,Census_SystemVolumeTotalCapacity,Census_HasOpticalDiskDrive,Census_TotalPhysicalRAM,Census_InternalPrimaryDiagonalDisplaySizeInInches,Census_InternalPrimaryDisplayResolutionHorizontal,Census_InternalPrimaryDisplayResolutionVertical,Census_InternalBatteryNumberOfCharges,Census_OSBuildNumber,Census_OSBuildRevision,Census_OSInstallLanguageIdentifier,Census_OSUILocaleIdentifier,Census_IsPortableOperatingSystem,Census_IsFlightsDisabled,Census_FirmwareManufacturerIdentifier,Census_FirmwareVersionIdentifier,Census_IsSecureBootEnabled,Census_IsVirtualDevice,Census_IsTouchEnabled,Census_IsPenCapable,Census_IsAlwaysOnAlwaysConnectedCapable,Wdft_IsGamer,Wdft_RegionIdentifier,HasDetections,EngineVersion_1.1.11701.0,EngineVersion_1.1.11804.0,EngineVersion_1.1.12400.0,EngineVersion_1.1.12805.0,EngineVersion_1.1.12902.0,EngineVersion_1.1.13000.0,EngineVersion_1.1.13102.0,EngineVersion_1.1.13103.0,EngineVersion_1.1.13202.0,EngineVersion_1.1.13303.0,EngineVersion_1.1.13407.0,EngineVersion_1.1.13504.0,EngineVersion_1.1.13601.0,EngineVersion_1.1.13701.0,EngineVersion_1.1.13704.0,EngineVersion_1.1.13804.0,EngineVersion_1.1.13902.0,EngineVersion_1.1.13903.0,EngineVersion_1.1.14001.0,EngineVersion_1.1.14002.0,EngineVersion_1.1.14003.0,EngineVersion_1.1.14103.0,EngineVersion_1.1.14104.0,EngineVersion_1.1.14201.0,EngineVersion_1.1.14202.0,EngineVersion_1.1.14303.0,EngineVersion_1.1.14305.0,EngineVersion_1.1.14306.0,EngineVersion_1.1.14405.2,EngineVersion_1.1.14500.2,EngineVersion_1.1.14500.5,EngineVersion_1.1.14600.4,EngineVersion_1.1.14700.3,EngineVersion_1.1.14700.4,EngineVersion_1.1.14700.5,EngineVersion_1.1.14800.1,EngineVersion_1.1.14800.3,EngineVersion_1.1.14901.3,EngineVersion_1.1.14901.4,EngineVersion_1.1.15000.1,EngineVersion_1.1.15000.2,EngineVersion_1.1.15100.1,EngineVersion_1.1.15200.1,EngineVersion_1.1.15300.5,EngineVersion_1.1.15300.6,AppVersion_4.10.14393.1066,AppVersion_4.10.14393.1198,AppVersion_4.10.14393.1532,AppVersion_4.10.14393.1593,AppVersion_4.10.14393.1613,AppVersion_4.10.14393.1794,AppVersion_4.10.14393.2248,AppVersion_4.10.14393.2273,AppVersion_4.10.14393.2457,AppVersion_4.10.14393.726,AppVersion_4.10.14393.953,AppVersion_4.10.205.0,AppVersion_4.10.209.0,AppVersion_4.11.15063.0,AppVersion_4.11.15063.1155,AppVersion_4.11.15063.447,AppVersion_4.12.16299.15,AppVersion_4.12.17007.17123,AppVersion_4.12.17007.18011,AppVersion_4.12.17007.18022,AppVersion_4.13.17134.1,AppVersion_4.13.17134.112,AppVersion_4.13.17134.191,AppVersion_4.13.17134.228,AppVersion_4.13.17134.319,AppVersion_4.14.17613.18038,AppVersion_4.14.17613.18039,AppVersion_4.14.17639.18041,AppVersion_4.15.17643.1000,AppVersion_4.15.17661.1001,AppVersion_4.16.17656.18051,AppVersion_4.16.17656.18052,AppVersion_4.17.17672.1000,AppVersion_4.17.17677.1000,AppVersion_4.17.17686.1003,AppVersion_4.18.1806.18062,AppVersion_4.18.1806.20015,AppVersion_4.18.1806.20021,AppVersion_4.18.1807.18072,AppVersion_4.18.1807.18075,AppVersion_4.18.1807.20063,AppVersion_4.18.1809.2,AppVersion_4.4.304.0,AppVersion_4.5.216.0,AppVersion_4.5.218.0,AppVersion_4.6.305.0,AppVersion_4.7.205.0,AppVersion_4.8.10240.16384,AppVersion_4.8.10240.17071,AppVersion_4.8.10240.17113,AppVersion_4.8.10240.17146,AppVersion_4.8.10240.17184,AppVersion_4.8.10240.17202,AppVersion_4.8.10240.17319,AppVersion_4.8.10240.17354,AppVersion_4.8.10240.17394,AppVersion_4.8.10240.17443,AppVersion_4.8.10240.17446,AppVersion_4.8.10240.17533,AppVersion_4.8.10240.17609,AppVersion_4.8.10240.17770,AppVersion_4.8.10240.17797,AppVersion_4.8.10240.17861,AppVersion_4.8.10

###  Разделим выборку на тренировочную и тестовую

P.S. в задачах классификации, как и в задаче регрессии, можно использовать технологию Кросс-Валидации. Например, по одному из двух следующих сценариев:

1) Отделить валидацию и тест, произвести подбор лучшей модели с помощью `K-Fold` на валидации, финально обучить выбранную модель на всей валидации и замерить качество на заранее отложенном финальном тесте!

2) Всю выборку назвать валидационной и на ней применить `K-Fold` без финального замера.

В этой домашней работе попросим Вас быть еще проще! :)
Реализуем просто технологию отложенной выборки в пропорции 3:1

In [15]:
### Разобьем выборку на трейн и тест
from sklearn.model_selection import train_test_split
# Your code is here
X = df_1.drop(columns='HasDetections')
Y = df_1['HasDetections']
x_train,x_test,y_train,y_test = train_test_split(X,Y,test_size=0.25,shuffle=True,random_state=1)

Соберите `Pipeline`, реализовав в нем 2 шага: стандартизация данных через `StandardScaler` и обучение логистической регрессии с помощью `LogisticRegression`, положите результаты в переменную `pipe`, а в классе модели `LogisticRegression` укажите параметр `penalty='none'`

In [16]:
### Здесь все просто и знакомо


from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline

pipe = Pipeline([('scaler', StandardScaler()), 
                 ('LR', LogisticRegression(penalty=None))])

Чтобы замерить качество работы такой модели на трейне и на тесте воспользуемся функцией `cross_validate`

Вопрос: что передавать ей в параметр cv? Ведь мы уже разделили нашу выборку на трейн и тест (имеем всего 1 fold). Для этого можно просто передать список от кортежа, содержащего индексы тренировочных и тестовых объектов.

In [17]:
# Your code is here
from sklearn.model_selection import cross_validate
import datetime



Что можете сказать про время работы алгоритма?

Очевидно, оно достаточно большое. Уж тем более для линейных моделей.

Такое как раз-таки происходит из-за того, что количество фичей, который мы передали нашей модели - гигантское! Классу требуется много времени и памяти, чтобы обработать датасет.

Поэтому те колонки, в которых количество уникальных категорий превышает какое-то адекватное число, следует кодировать иначе, нежели с помощью технологии `One-Hot-Encoding`.

Теперь вы верите, что более умные кодировки зачастую прям необходимы! Раньше мы этот факт не демонстрировали!

Дело еще вот в чем: в классе `LogisticRegression`, как и, например, `Lasso`, есть параметр, ограничивающий максимальное количество итераций во время обучения модели. Так, если данных много и итераций тоже ожидается большое число, найденная разделяющая гиперплоскость может оказаться не самой лучшей, так как наш алгоритм (будь то градиентный спуск или любой иной) просто 'не доползет'. 

#### Задание 2

Теперь попробуем другой метод кодирования категориальных колонок, а именно счётчики.
Построем ту же модель и на том же разделении, просто заново иначе переобработаем датасет. 

Для тех категориальных признаков, у которых количество уникальных значений в колоночках больше 5, применим `MeanTargetEncoding`.

Для остальных оставим любимый `OneHotEncoding` (как делали на практике и в предыдущем уроке).

In [18]:
### Закодируем категории с большим кол-ом уникальных значений с помощью счетчиков
### Закрывая глаза на возможную проблему переобучения
### То есть будем считать средние по всему датасету
for col in cat_col:
    if df_2[col].nunique() < 5:
        one_hot = pd.get_dummies(df_2[col], prefix=col, drop_first=True)
        df_2 = pd.concat((df_2.drop(col, axis=1), one_hot), axis=1)
        
    else:
        mean_target = df_2.groupby(col)['HasDetections'].mean()
        df_2[col] = df_2[col].map(mean_target)


# Your code is here

In [19]:
X_2 = df_2.drop(columns=['HasDetections'])
y_2 = df_2['HasDetections']

Опять обучим модель, пока что без изменений! Скажите, стало ли быстрее? А что с качеством?

In [20]:
### Заново обучим модель и провалидируем
### Очевидно стало супер-быстрее!
from sklearn.model_selection import cross_validate
import datetime
begin_time = datetime.datetime.now()
x_train,x_test,y_train,y_test = train_test_split(X_2,y_2,test_size=0.25,random_state=1,shuffle=True)
cv_custom = [(x_train.index.to_list(),x_test.index.to_list())]
cv_results = cross_validate(pipe,X_2,y_2,scoring='accuracy',cv=cv_custom,return_train_score=True)
print(f"Accuracy на трейне: {np.mean(cv_results['train_score']).round(3)}")
print(f"Accuracy на тесте: {np.mean(cv_results['test_score']).round(3)}")

print(f"Время работы алгоритма: {datetime.datetime.now() - begin_time}")

Accuracy на трейне: 0.638
Accuracy на тесте: 0.64
Время работы алгоритма: 0:00:02.206645


#### Задание 3: Регуляризация

Как и в моделях регрессии, решая задачу классификации, можем штрафовать минимизируемый функционал за большие веса, добавив к нему L1 или L2 норму весов (все как раньше!).

Для этого в изначальном классе `LogisticRegression` изменить параметр `penalty` на l1 или l2 соответственно. Выберите второй вариант! Можно воспользоваться методом `set_params` и применить его к `pipe`.

In [21]:
### Установим в качестве параметра penalty
### L2 регуляризатор!
from sklearn.model_selection import GridSearchCV
pipe.set_params(LR__penalty='l2')
alphas = np.linspace(0.01,100,100)
param_grid ={
    'LR__C': alphas
}


search_alpha = GridSearchCV(pipe, param_grid, 
                            cv=cv_custom, scoring='accuracy')

search_alpha.fit(X_2, y_2)

print(f"Best parameter (CV score={search_alpha.best_score_:.5f}):")
print(search_alpha.best_params_)
# Your code

Best parameter (CV score=0.63964):
{'LR__C': np.float64(0.01)}


In [22]:
df_2.to_csv('proccessed_train.csv',index=False)